In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Connection

In [0]:
storage_account ='olympics20250330'
application_id = '' #this one comes in the Microsoft entry id app
directory_or_tenant_id = '' #this one comes in the Microsoft entry id app
secret_value_or_service_credential = '' #this one comes in the Microsoft entry id app


In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", secret_value_or_service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_or_tenant_id}/oauth2/token")

### Data Ingestion

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load('abfss://olympic-data@olympics20250330.dfs.core.windows.net/raw_data/Athletes.csv')
coaches  = spark.read.format("csv").option("header","true").option("inferSchema","true").load('abfss://olympic-data@olympics20250330.dfs.core.windows.net/raw_data/Coaches.csv')
entriesgender  = spark.read.format("csv").option("header","true").option("inferSchema","true").load('abfss://olympic-data@olympics20250330.dfs.core.windows.net/raw_data/EntriesGender.csv')
medals  = spark.read.format("csv").option("header","true").option("inferSchema","true").load('abfss://olympic-data@olympics20250330.dfs.core.windows.net/raw_data/Medals.csv')
teams   = spark.read.format("csv").option("header","true").option("inferSchema","true").load('abfss://olympic-data@olympics20250330.dfs.core.windows.net/raw_data/Teams.csv')

     

#### Testing Transformations

In [0]:
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold")

top_gold_medal_countries.display()

TeamCountry,Gold
United States of America,39
People's Republic of China,38
Japan,27
Great Britain,22
ROC,20
Australia,17
Netherlands,10
France,10
Germany,10
Italy,10


In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.display()

Discipline,Female,Male,Total,Avg_Female,Avg_Male
3x3 Basketball,32,32,64,0.5,0.5
Archery,64,64,128,0.5,0.5
Artistic Gymnastics,98,98,196,0.5,0.5
Artistic Swimming,105,0,105,1.0,0.0
Athletics,969,1072,2041,0.4747672709456149,0.5252327290543851
Badminton,86,87,173,0.49710982658959535,0.5028901734104047
Baseball/Softball,90,144,234,0.38461538461538464,0.6153846153846154
Basketball,144,144,288,0.5,0.5
Beach Volleyball,48,48,96,0.5,0.5
Boxing,102,187,289,0.35294117647058826,0.6470588235294118


### Save Data to Parquet format

In [0]:
arr_dfs = [athletes, coaches, entriesgender, medals, teams]
arr_folder_names = ['athletes', 'coaches', 'entriesgender', 'medals', 'teams']

zipped = list(zip(arr_dfs, arr_folder_names))

for item in zipped:
    item[0].write.mode('overwrite').parquet(f"abfss://olympic-data@olympics20250330.dfs.core.windows.net/transformed_data/{item[1]}/")
     